# 检查内存和GPU环境

In [1]:
import psutil
import torch


ram = psutil.virtual_memory()
ram_total = ram.total / (1024**3)
print('RAM: %.2f GB' % ram_total)

print('=============GPU INFO=============')
if torch.cuda.is_available():
  !/opt/bin/nvidia-smi || ture
else:
  print('GPU NOT available')

RAM: 12.67 GB
=============GPU INFO=============
Tue May 21 03:32:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+------------------

# 安装Xinference依赖

In [2]:
%pip install -U -q xinference[transformers] openai langchain langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip show xinference

Name: xinference
Version: 0.11.1
Summary: Model Serving Made Easy
Home-page: https://github.com/xorbitsai/inference
Author: Qin Xuye
Author-email: qinxuye@xprobe.io
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: aioprometheus, async-timeout, click, fastapi, fsspec, gradio, huggingface-hub, modelscope, openai, opencv-contrib-python, passlib, peft, pillow, pydantic, pynvml, python-jose, requests, s3fs, sse-starlette, tabulate, timm, torch, tqdm, typer, typing-extensions, uvicorn, xoscar
Required-by: 


# 开始demo演示
## 使用如下命令本地化运行Xinference

In [4]:
!nohup xinference-local  > xinference.log 2>&1 &

默认的IP：127.0.0.1和端口9997
一旦Xinference运行，我们就可以通过web UI, cURL, 命令行, Xinference python客户端进行访问了

In [ ]:
# 从transformers库下载模型
!xinference-local -H 0.0.0.0

# 从modelscope下载模型
!XINFERENCE_MODEL_SRC=modelscope xinference-local -H 0.0.0.0

**命令行工具是**：xinference

In [5]:
!xinference --help

Usage: xinference [OPTIONS] COMMAND [ARGS]...

  Xinference command-line interface for serving and deploying models.

Options:
  -v, --version       Show the current version of the Xinference tool.
  --log-level TEXT    Set the logger level. Options listed from most log to
                      least log are: DEBUG > INFO > WARNING > ERROR > CRITICAL
                      (Default level is INFO)
  -H, --host TEXT     Specify the host address for the Xinference server.
  -p, --port INTEGER  Specify the port number for the Xinference server.
  --help              Show this message and exit.

Commands:
  chat           Chat with a running LLM.
  engine         Query the applicable inference engine by model name.
  generate       Generate text using a running LLM.
  launch         Launch a model with the Xinference framework with the...
  list           List all running models in Xinference.
  login          Login when the cluster is authenticated.
  register       Register a new model wit

# 运行Qwen-Chat
Xinference支持一系列LLMs，地址： https://inference.readthedocs.io/en/latest/models/builtin/.

下面我们使用内置的Qwen-1_8B-Chat来进行演示

使用参数--model-uid或者-u来指定模型生成UID，如果不指定，模型是自动生成。

这里我们指定为：my-llm

In [6]:
!xinference launch -u my-llm -n qwen-chat -s 1_8 -f pytorch -en transformers

Launch model name: qwen-chat with kwargs: {}
Model uid: my-llm


# 交互式运行模型

## 1）使用OpenAI兼容的方式：

In [7]:
import openai

messages=[
    {
        "role": "user",
        "content": "Who are you?"
    }
]

client = openai.Client(api_key="empty", base_url=f"http://0.0.0.0:9997/v1")
client.chat.completions.create(
    model="my-llm",
    messages=messages,
)

ChatCompletion(id='chat1c46cbd6-1725-11ef-978a-0242ac1c000c', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am an artificial intelligence language model named QianWen, developed by Alibaba Cloud. I am designed to answer questions and provide information to users, as well as engage in conversations with them using natural language processing technology. How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1716263362, model='my-llm', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=51, prompt_tokens=23, total_tokens=74))

## 2）通过curl发送请求

In [8]:
!curl -k -X 'POST' -N \
  'http://127.0.0.1:9997/v1/chat/completions' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{ "model": "my-llm", "messages": [ {"role": "system", "content": "You are a helpful assistant." }, {"role": "user", "content": "What is the largest animal?"} ]}'

{"id":"chat49b1d52a-1725-11ef-b2b5-0242ac1c000c","object":"chat.completion","created":1716263438,"model":"my-llm","choices":[{"index":0,"message":{"role":"assistant","content":"The largest animal on Earth is the blue whale, which can grow up to 100 feet (30 meters) long and weigh up to 200 tons (180 metric tonnes). It has the most powerful muscles in any living creature and can dive deep into the ocean to hunt for prey. Blue whales feed primarily on krill and other small marine animals and can be found in all of the world's oceans, although they are most commonly seen in the waters off the coast of Antarctica."},"finish_reason":"stop"}],"usage":{"prompt_tokens":25,"completion_tokens":103,"total_tokens":128}}

## 3）使用Xinference python客户端

In [9]:
from xinference.client import RESTfulClient
client = RESTfulClient("http://127.0.0.1:9997")
model = client.get_model("my-llm")
model.chat(
    prompt="hello",
    chat_history=[
    {
        "role": "user",
        "content": "What is the largest animal?"
    }]
)

{'id': 'chat70028f62-1725-11ef-b2b5-0242ac1c000c',
 'object': 'chat.completion',
 'created': 1716263503,
 'model': 'my-llm',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "Hello! How can I help you today? Is there something specific you would like to know or discuss? I'm here to provide information and answer any questions you might have."},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 31, 'completion_tokens': 35, 'total_tokens': 66}}

## 4）与LangChain集成

In [10]:
from langchain.llms import Xinference
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = Xinference(server_url='http://127.0.0.1:9997', model_uid='my-llm')

template = 'What is the largest {kind} on the earth?'

prompt = PromptTemplate(template=template, input_variables=['kind'])

llm_chain = LLMChain(prompt=prompt, llm=llm)

generated = llm_chain.run(kind='plant')
print(generated)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 The largest plant on Earth is likely to be a very large tree, such as a giant sequoia or a ponderosa pine. These trees can grow up to 300 feet tall and can weigh hundreds of tons.
There are many examples of very large trees in the world, but it's difficult to say which one is the largest. One tree that comes to mind is the Grand Canyon Tree, which is over 45 feet tall and weighs about 120 tons. It was discovered in the Arizona desert in 1978 and has since become an iconic symbol of the canyon and the natural wonders of North America.
However, there are also other extremely large trees in the world, such as the Redwood Tree in California, which is over 1,000 feet tall and can weigh over 2 million tons. And there are even giant trees that are found in other parts of the world, such as the Ginkgo biloba tree in China, which can grow up to 60 feet tall and can weigh over 40,000 pounds.
